In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from itertools import chain
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select

In [24]:

driver= webdriver.Chrome()


In [25]:
driver.get("https://www.trademap.org/Index.aspx")

In [26]:
driver.maximize_window()
waits= WebDriverWait(driver, 20)

## Login Users

In [27]:
login=waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_MenuControl_Label_Login')))
login.click()
getemail= waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#Username')))
getemail.send_keys('homa.abedinzadeh@gmail.com')

getpassword= waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#Password')))
getpassword.send_keys('Aa@97019701')

btnlogin=waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.container > div > div:nth-child(2) > div > div > div > form > fieldset > div:nth-child(4) > div > button'))).click()



## Get Codes and Country

In [28]:
inputproduct= waits.until(EC.element_to_be_clickable ((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_Input')))
inputproduct.send_keys(' 170410')

clkinput= waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR,  '#ctl00_PageContent_RadComboBox_Product_c0'))).click()


inputContry= waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR,  '#ctl00_PageContent_RadComboBox_Country_Input')))
inputContry.send_keys('Iran, Islamic Republic of')

clkinput2= waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR,  '#ctl00_PageContent_RadComboBox_Country_c0'))).click()

btn=waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Button_TradeIndicators'))).click()


In [29]:
incshow=waits.until(EC.element_to_be_clickable((By.XPATH , "//select[@id='ctl00_PageContent_GridViewPanelControl_DropDownList_PageSize']")))
drop=Select(incshow)
drop.select_by_value('300')